In [1]:
## Pasos:

#1. RandomForest - MDI
#2. Seleccionar mejores caractéristicas.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.base import clone
plt.style.use('ggplot')
pd.set_option("display.max_columns", 50)

%matplotlib inline

### 1. Leer datos pre-procesados

In [3]:
#Lectura del dataset
file_name = "MI_database.csv"

X_train =  pd.read_csv(
    'data/02_data_intermedia/X_MI_entrenamiento_preprocesado.csv', 
    header=0, 
    sep=","
)
X_test =  pd.read_csv(
    'data/02_data_intermedia/X_MI_pruebas_preprocesado.csv', 
    header=0, 
    sep=","
)
y_train =  pd.read_csv(
    'data/02_data_intermedia/y_MI_entrenamiento_preprocesado.csv', 
    header=0, 
    sep=","
)
y_test =  pd.read_csv(
    'data/02_data_intermedia/y_MI_pruebas_preprocesado.csv', 
    header=0, 
    sep=","
)

### 2. Selección de mejores características usando MDI y RandomForest

In [4]:
rf = RandomForestRegressor(80, n_jobs=-1, oob_score=True)
#  40, n_jobs=-1, oob_score=True
rf.fit(X_train, y_train)

<ipython-input-4-b21a45ef4026>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor(n_estimators=80, n_jobs=-1, oob_score=True)

In [5]:
rf.feature_importances_

array([6.53119588e-07, 9.43628372e-01, 2.83252555e-02, 1.93867656e-02,
       5.50641334e-03, 1.26678113e-03, 8.10599032e-04, 6.37919081e-05,
       9.90542723e-05, 2.62249016e-05, 4.62846930e-06, 8.08751004e-07,
       3.98560029e-05, 1.33122985e-05, 2.90161124e-06, 2.01328666e-06,
       8.68789261e-07, 3.04889430e-06, 9.52453404e-07, 1.02272927e-05,
       2.43158226e-08, 4.74638126e-09, 1.79452500e-09, 0.00000000e+00,
       3.17424948e-08, 4.95820303e-10, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.90712729e-07,
       8.07003630e-07, 0.00000000e+00, 3.11542737e-06, 2.64870452e-06,
       0.00000000e+00, 0.00000000e+00, 6.77075995e-10, 2.92119769e-05,
       9.99029128e-06, 3.63863311e-07, 1.03542719e-06, 1.02500131e-06,
       8.27652749e-08, 2.14983509e-08, 1.33620883e-08, 9.01731779e-07,
       1.33665076e-05, 3.34118329e-05, 3.38601694e-07, 4.44547269e-07,
      

In [6]:
importances = pd.DataFrame({"columns":X_train.columns, "importances":rf.feature_importances_}).sort_values("importances", ascending=False)

importances.style.bar()

,columns,importances
1,FIBR_PREDS,0.943628
2,PREDS_TAH,0.028325
3,JELUD_TAH,0.019387
4,FIBR_JELUD,0.005506
5,A_V_BLOK,0.001267
6,OTEK_LANC,0.000811
89,ALT_BLOOD,0.000118
74,n_p_ecg_p_05,0.000100
8,DRESSLER,0.000099
68,n_r_ecg_p_08,0.000088


### <u>Nota:</u> 
<mark>En este experimento se considera valores mayores a 0.009 porque ratios de importancia menores no aportan o 
aportan muy poco al modelo. Adicionalmente, se consideró este ratio de corte con el fin de no perder
información significativa para el modelo. </mark>

In [17]:
columnas_importantes = importances.loc[importances.importances>=0.0000009,"columns"].unique()
X_train_caracterizado = X_train[columnas_importantes]
X_test_caracterizado = X_test[columnas_importantes]

In [18]:
print(f"El proceso de selección de características obtiene un total de {len(columnas_importantes)} características.")

El proceso de selección de características obtiene un total de 51 características.


In [19]:
X_train_caracterizado.to_csv('data/03_entrada_modelo/MI_caracterizado.csv')

### 3. Guardar resultados de la selección de características

In [20]:
X_train_caracterizado.to_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv',index=False)
X_test_caracterizado.to_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv',index=False)

y_train.to_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv',index=False)
y_test.to_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv',index=False)